# Task

Write a function `cut_suffix` which takes a string and a suffix. A function should return this string
without the given suffix.

```python
cut_suffix("foobar", "bar")
>>> "foo"

cut_suffix("foobar", "boo")
>>> "foobar"
```

In [ ]:
# Your work here

def cut_suffix(sentence: str, suffix: str) -> str:
    return sentence.removesuffix(suffix)

In [ ]:
cut_suffix("foobar", "bar")

'foo'

In [ ]:
cut_suffix("foobar", "boo")

'foobar'

Write a function `boxed` which takes a string and two arguments: a symbol `fill` and a number
`pad`. A result of the `boxed` function execution should be a string surrounded by `fill` symbols as
it’s shown in the example.

```python
print(boxed("Hello world", fill="*", pad=2))
print(boxed("Fishy", fill="#", pad=1))
```

result:

```md
*****************
** Hello world **
*****************

#########
# Fishy #
#########
```

In [ ]:
def boxed(sentence: str, fill: str = '#', pad: int = 1):
    return (fill*(pad*2 + 2 + len(sentence))) + '\n' \
        + ((fill*pad) + ' ' + sentence + ' ' + (fill*pad)) + '\n' \
        + (fill*(pad*2 + 2 + len(sentence)))

In [ ]:
print(boxed("Hello world", fill="*", pad=2))
print(boxed("Fishy", fill="#", pad=1))

*****************
** Hello world **
*****************
#########
# Fishy #
#########



---



**She-bang** – a sequence `#!` which is used in Unix-like systems to run executable scripts. **She-bang**
is always written on the first line in the script. After **she-bang** there is path to an interpreter program
written, for example:

`#! /bin/sh`

`#!/usr/bin/env python -v`

> Look at more example of she-bang here: http://en.wikipedia.org/wiki/Shebang_(Unix)


Write a function `parse_shebang` which takes a path to an executable script and return a path to an
interpreter program, if a script contains `she-bang` and None otherwise.
For the scripts in the example above:
```python
parse_shebang("./example1.txt")
>>> "/bin/sh"

parse_shebang("./example2.txt")
>>> "/usr/bin/env python -v"
```

In [ ]:
def parse_shebang(path: str):
    with open(path, mode='r') as file:
        first_line = file.readline()
        if first_line.startswith('#!'):
            index = 2
            while True:
                if first_line[index] == '/':
                    break
                index += 1
            return first_line[index:]
    
        else:
            return None

In [ ]:
parse_shebang("./example1.txt")

'/bin/sh'

In [ ]:
parse_shebang("./example2.txt")

'/usr/bin/env python -v'

# Special Task (Bonus 4%)

A probabilistic langauge model describes pieces of text of some language in terms of random processes.
One of the simplest language model can be stated the following way. Let’s assume that we know a set of all
words in a language. Let’s generate words in a sentence from left to right one-by-one:

- Randomly take first two words from the set of all words.
- Each i’th word we will generate having from two previous (i - 1)’th and (i - 2)’th words.

Let’s try to build a language model based on lyrics of Taylor Swift's songs!

1. Write a function `words` which takes a text file and returns a list of words from a file:
```python
import io
handle = io.StringIO("""Can we always be this close forever and ever?
And ah, take me out, and take me home forever and ever.""")
words(handle)
>>>['Can', 'we', 'always', 'be', 'this', 'close', 'forever', 'and', 'ever?\n', 'And', 'ah,', 'take', 'me', 'out,', 'and', 'take', 'me', 'home', 'forever', 'and', 'ever.',
 ]
```

**Mind that punctuation and new-line characters stay unchanged!!!**

In [ ]:
import io
handle = io.StringIO("""Can we always be this close forever and ever?
And ah, take me out, and take me home forever and ever.""")

def words(handle: io.StringIO) -> list[str]:
    text = list()
    while True:
        line = handle.readline()
        if line == '':
            break
        text.extend(line.split(' '))
    return text

print(words(handle))

['Can', 'we', 'always', 'be', 'this', 'close', 'forever', 'and', 'ever?\n', 'And', 'ah,', 'take', 'me', 'out,', 'and', 'take', 'me', 'home', 'forever', 'and', 'ever.']


---

2. Write a function `transition_matrix` which takes a list of words and returns a dictionary. This
dictionary for every pair of words `(u, v)` contains a list of words `w` which follow words `u` and `v` in the
input list of words. For the example above:

```python
language = words(handle)
m = transition_matrix(language)
m[("take", "me")]
>>> ["out,", "home"]

m[("we", "always")]
>>> ["be"]

m[("forever", "and")]
>>> ["ever?\n", "ever."]
```

In [ ]:
# It will contain duplicate words
def transition_matrix(word_list: list[str]) -> dict:
    result = dict()
    for index in range(len(word_list)-2):
        word_pair = (word_list[index], word_list[index + 1])
        if word_pair in result.keys():
            result[word_pair].append(word_list[index + 2])
        else:
            result[word_pair] = [word_list[index + 2]]
    return result

In [ ]:
import io
handle = io.StringIO("""Can we always be this close forever and ever?
And ah, take me out, and take me home forever and ever.""")

language = words(handle)
m = transition_matrix(language)

In [ ]:
m[("take", "me")]

['out,', 'home']

In [ ]:
m[("we", "always")]

['be']

In [ ]:
m[("forever", "and")]

['ever?\n', 'ever.']

---

3. Write a function `markov_chain` which generates sentences of a defined size. A function takes three
parameters:
- a list of words, a result `words` function execution,
- a dictionary, built with `transition_matrix` function,
- an integer – a number of words in sentence to be generated.


Let me remind how to generate random sentences. Let’s generate words in a sentence from left to right
one-by-one:
- Randomly take two first words from all words list words.
- Each `i`’th word will be generated using previous two `(i - 1)`’th and `(i - 2)`’th words (with help of `transition_matrix`).
- If this pair didn’t happen to exist (it’s in `transition_matrix` dictionary) then `i`’th word is taken randomly from the set of all
words.

You will need functions `random.randint` and `random.choice`.

In [ ]:
import random

def markov_chain(words: list[str], transition_matrix: dict[tuple[str, str]: list[str]], number: int) -> str:
    result = list()
    result.append(random.choice(words))
    result.append(random.choice(words))
    for index in range(2, number):
        if (result[index - 1], result[index - 2]) in transition_matrix.keys():
            result.append(random.choice(transition_matrix[(result[index - 1], result[index - 2])]))
        else:
            result.append(random.choice(words))
    return ' '.join(result)

---

4. Write a function `taylor_swifter()` which takes a path to a file `taylor_swift.txt` and an
integer – a length of a sentence and returns a sentence of specified language on Taylor Swift's
language.

```python
print(taylor_swifter("./taylor_swift.txt", 30))

>>>'well dancing pack I got this music in my head tell me to the garden? In the garden, would you trust me If I told you it was never mine'
```

In [ ]:
def taylor_swifter(path: str, sentence_length: int) -> str:
    handle = open(path, 'r')
    word_list = words(handle)
    handle.close()

    tf = transition_matrix(word_list)

    return markov_chain(word_list, tf, sentence_length)

In [ ]:
print(taylor_swifter('./Taylor_Swift.txt', 100))

ever live forever
 days
 overdramatic it dancing what band how like jeans you Because singing time learned forever Was me
 (oh)
 you, And a You I and said
 rules Methodist
 queens
 shake We you'll never you
 right good been?
 drunk will fancy I I'm you
 drunk feet
 over, a Would all my go ever)
 We and that's on middle I chances
 find
 Your I'm the she too getting my your But So take was were seventeen, queen
 Oh, I'm was plans
 if Sometimes with damn shake, And home
 'round can If we leads, free me thing to my to house



>The lyrics came from well-known taylor swifts songs and use for education only.